In [2]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt

In [3]:
DATA_ROOT_DIR = "FashionMNIST"

In [4]:
train_data = torchvision.datasets.FashionMNIST(root=DATA_ROOT_DIR, train=True, download=True, transform=torchvision.transforms.ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
test_data = torchvision.datasets.FashionMNIST(root=DATA_ROOT_DIR, train=False, download=True, transform=torchvision.transforms.ToTensor())

In [6]:
BATCH_SIZE = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [7]:
for X,y in test_data_loader:
  print(X.shape, X.dtype)
  print(y.shape, y.dtype,torch.unique(y)) #gives number of output classes
  break

torch.Size([64, 1, 28, 28]) torch.float32
torch.Size([64]) torch.int64 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu" 
device

'cuda'

In [9]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()

    self.flatten = nn.Flatten()

    self.learner_relu_stack = nn.Sequential(nn.Linear(28*28,512), nn.ReLU(),
                                            nn.Linear(512,512), nn.ReLU(),
                                            nn.Linear(512,10), nn.ReLU())
    
  def forward(self,X):
    X=self.flatten(X)
    logits = self.learner_relu_stack(X)
    return logits


In [10]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (learner_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [19]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)

  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch%100==0 :
      loss, current = loss.item(), batch*len(X)
      print(f"Training Loss: {loss} [{current}/{size}]")

In [24]:
def test(dataloader, model):
  size = len(dataloader.dataset)
  model.eval()

  test_loss, correct = 0,0

  with torch.no_grad():
     for batch, (X,y) in enumerate(dataloader):
       X,y = X.to(device), y.to(device)
       pred = model(X)
       test_loss = test_loss + loss_fn(pred,y).item()
       correct = correct + (pred.argmax(1)==y).type(torch.float).sum().item()

  test_loss = test_loss/size
  correct = correct/size

  print(f" Test Loss: {100*correct}, average loss: {test_loss}") 

In [25]:
EPOCHS = 3
for epoch in range(EPOCHS):
  print(f" EPOCH: {epoch+1} \n------------------------------")

  train(train_data_loader, model, loss_fn, optimizer)
  test(test_data_loader, model)

 EPOCH: 1 
------------------------------
Training Loss: 2.242020606994629 [0/60000]
Training Loss: 2.2028543949127197 [6400/60000]
Training Loss: 2.1967732906341553 [12800/60000]
Training Loss: 2.1873362064361572 [19200/60000]
Training Loss: 2.140460252761841 [25600/60000]
Training Loss: 2.185962200164795 [32000/60000]
Training Loss: 2.14082670211792 [38400/60000]
Training Loss: 2.1529128551483154 [44800/60000]
Training Loss: 2.188880681991577 [51200/60000]
Training Loss: 2.0958685874938965 [57600/60000]
 Test Loss: 29.360000000000003, average loss: 0.03334743349552154
 EPOCH: 2 
------------------------------
Training Loss: 2.1732027530670166 [0/60000]
Training Loss: 2.1008129119873047 [6400/60000]
Training Loss: 2.087664842605591 [12800/60000]
Training Loss: 2.070314645767212 [19200/60000]
Training Loss: 1.9888805150985718 [25600/60000]
Training Loss: 2.046088457107544 [32000/60000]
Training Loss: 2.0115392208099365 [38400/60000]
Training Loss: 2.0257668495178223 [44800/60000]
Train

In [26]:
torch.save(model, "model.pth")

In [31]:
classes = [ " T-shirt/top", "Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

model.eval()
model.to("cpu")

X,y = test_data[0][0], test_data[0][1]

with torch.no_grad():
  pred = model(X)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(predicted, actual)


Ankle boot Ankle boot
